<a href="https://colab.research.google.com/github/MitalPattani/Sales-Accuracy-Prediction/blob/master/sales_prediction_accuracy(data_visualization).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
from plotly.subplots import make_subplots
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [ ]:
calendar = pd.read_csv("./gdrive/My Drive/sales_prediction_accuracy/calendar.csv")
sales_data = pd.read_csv("./gdrive/My Drive/sales_prediction_accuracy/sales_train_validation.csv")
sell_price = pd.read_csv("./gdrive/My Drive/sales_prediction_accuracy/sell_prices.csv")

In [ ]:
sales_data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,...,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,3,1,3,1,2,2,0,1,1,1,1,0,0,0,0,0,1,0,4,2,3,0,1,2,0,0,0,1,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,2,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,3,4,2,1,4,1,3,5,0,6,6,0,0,0,0,3,1,2,1,3,1,0,2,5,4,2,0,3,0,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,3,2,2,2,3,1,0,0,0,0,1,0,4,4,0,1,4,0,1,0,1,0,1,1,2,0,1,1,2,1,1,0,1,1,2,2,2,4


In [ ]:
sales_data['cat_id'].value_counts()

FOODS        14370
HOUSEHOLD    10470
HOBBIES       5650
Name: cat_id, dtype: int64

In [ ]:
cat_sales = sales_data.copy()
cat_sales = cat_sales.groupby(by=['state_id','cat_id'], as_index = False).mean()
cat_sales['state_cat'] = cat_sales['state_id']+'_'+cat_sales['cat_id']
cat_sales.head()
cat_sales = cat_sales.drop(['state_id','cat_id'],axis=1)
cat_sales = cat_sales.T
cat_sales.columns = cat_sales.loc['state_cat']
cat_sales = cat_sales.drop('state_cat')

In [ ]:
cat_sales_holiday = pd.merge(cat_sales,calendar, left_index = True, right_on = 'd')
cat_sales_holiday = cat_sales_holiday.drop(['date','wm_yr_wk','weekday','wday','month','year','snap_CA','snap_TX','snap_WI'], axis = 1)
cat_sales_holiday.at[cat_sales_holiday['event_name_1'].notna() | cat_sales_holiday['event_name_1'].notna(),'is_holiday'] = 1
cat_sales_holiday.at[cat_sales_holiday['event_name_1'].isna() & cat_sales_holiday['event_name_1'].isna(),'is_holiday'] = 0
cat_sales_holiday['is_holiday'] = cat_sales_holiday['is_holiday'].astype(int)
cat_sales_holiday['d'] = cat_sales_holiday.d.str.lstrip('d_')

In [ ]:
cat_sales_holiday.head()

,CA_FOODS,CA_HOBBIES,CA_HOUSEHOLD,TX_FOODS,TX_HOBBIES,TX_HOUSEHOLD,WI_FOODS,WI_HOBBIES,WI_HOUSEHOLD,d,event_name_1,event_type_1,event_name_2,event_type_2,is_holiday
0,1.75731,0.797345,0.547278,1.58965,0.518584,0.543139,1.44375,0.638938,0.538364,1,NaN,NaN,NaN,NaN,0
1,1.71573,0.690708,0.568768,1.63071,0.513274,0.55078,1.36071,0.546313,0.484559,2,NaN,NaN,NaN,NaN,0
2,1.20807,0.651327,0.404011,1.18859,0.310324,0.359121,1.18441,0.40354,0.352436,3,NaN,NaN,NaN,NaN,0
3,1.36813,0.621681,0.424546,1.26885,0.477286,0.350844,1.28601,0.268437,0.313594,4,NaN,NaN,NaN,NaN,0
4,1.24878,0.522566,0.373926,1.0675,0.295575,0.257561,0.654836,0.0778761,0.112703,5,NaN,NaN,NaN,NaN,0


In [ ]:
CA_FOODS = pd.DataFrame({'Data':cat_sales_holiday['CA_FOODS'] , 'd':cat_sales_holiday['d'], 'is_holiday': cat_sales_holiday['is_holiday']})
CA_FOODS = CA_FOODS[CA_FOODS['Data'] > 0.5]

CA_HOBBIES = pd.DataFrame({'Data':cat_sales_holiday['CA_HOBBIES'] , 'd':cat_sales_holiday['d'], 'is_holiday': cat_sales_holiday['is_holiday']})
CA_HOBBIES = CA_HOBBIES[CA_HOBBIES['Data'] > 0]

CA_HOUSEHOLD = pd.DataFrame({'Data':cat_sales_holiday['CA_HOUSEHOLD'] , 'd':cat_sales_holiday['d'], 'is_holiday': cat_sales_holiday['is_holiday']})
CA_HOUSEHOLD= CA_HOUSEHOLD[CA_HOUSEHOLD['Data'] > 0]

TX_FOODS = pd.DataFrame({'Data':cat_sales_holiday['TX_FOODS'] , 'd':cat_sales_holiday['d'], 'is_holiday': cat_sales_holiday['is_holiday']})
TX_FOODS = TX_FOODS[TX_FOODS['Data'] > 0.5]
TX_FOODS = TX_FOODS[TX_FOODS['Data'] < 3.0]

TX_HOBBIES = pd.DataFrame({'Data':cat_sales_holiday['TX_HOBBIES'] , 'd':cat_sales_holiday['d'], 'is_holiday': cat_sales_holiday['is_holiday']})
TX_HOBBIES = TX_HOBBIES[TX_HOBBIES['Data'] > 0]

TX_HOUSEHOLD = pd.DataFrame({'Data':cat_sales_holiday['TX_HOUSEHOLD'] , 'd':cat_sales_holiday['d'], 'is_holiday': cat_sales_holiday['is_holiday']})
TX_HOUSEHOLD = TX_HOUSEHOLD[TX_HOUSEHOLD['Data'] > 0] 

WI_FOODS = pd.DataFrame({'Data':cat_sales_holiday['WI_FOODS'] , 'd':cat_sales_holiday['d'], 'is_holiday': cat_sales_holiday['is_holiday']})
WI_FOODS = WI_FOODS[WI_FOODS['Data'] > 0.5]

WI_HOBBIES = pd.DataFrame({'Data':cat_sales_holiday['WI_HOBBIES'] , 'd':cat_sales_holiday['d'], 'is_holiday': cat_sales_holiday['is_holiday']})
WI_HOBBIES = WI_HOBBIES[WI_HOBBIES['Data'] > 0]

WI_HOUSEHOLD = pd.DataFrame({'Data':cat_sales_holiday['WI_HOUSEHOLD'] , 'd':cat_sales_holiday['d'], 'is_holiday': cat_sales_holiday['is_holiday']})
WI_HOUSEHOLD = WI_HOUSEHOLD[WI_HOUSEHOLD['Data'] > 0.05]

categories = [CA_FOODS, CA_HOBBIES, CA_HOUSEHOLD, TX_FOODS, TX_HOBBIES, TX_HOUSEHOLD, WI_FOODS, WI_HOBBIES, WI_HOUSEHOLD]
categories_name = ('CA_FOODS', 'CA_HOBBIES', 'CA_HOUSEHOLD', 'TX_FOODS', 'TX_HOBBIES', 'TX_HOUSEHOLD', 'WI_FOODS', 'WI_HOBBIES', 'WI_HOUSEHOLD')


In [ ]:
fig = make_subplots(rows=3, cols=3, subplot_titles = categories_name)
i = 1
j = 1
for ind, val in enumerate(categories):
    if j > 3:
        j = 1
        i = i+1
        fig.add_trace(
            go.Scatter(x= val['d'], y=val['Data']),
            row=i, col=j
        )
    else:
        fig.add_trace(
            go.Scatter(x= val['d'], y=val['Data']),
            row=i, col=j
        )
    j = j+1

fig.update_layout(height=600, width=800, title_text="Total mean sales")
fig.show()
